<a href="https://colab.research.google.com/github/SveinungOverland/ZeroGO/blob/master/NN/DCNN_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Convolutional Neural Net for ZeroGO V1

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

tf.__version__

'2.0.0'

## Layers and heads

### Residual layer

In [0]:
class Residual(layers.Layer):
  def __init__(self, **kwargs):
    super(Residual, self).__init__(**kwargs)
    self.conv1 = layers.Conv2D(256, kernel_size=(3,3))
    self.norm1 = layers.BatchNormalization()
    self.relu1 = layers.ReLU()
    self.conv2 = layers.Conv2D(256, kernel_size=(3,3))
    self.norm2 = layers.BatchNormalization()
    self.relu2 = layers.ReLU()
    self.add = layers.Add()
    self.relu3 = layers.ReLU()
    
  def build(self, input_shape):
    # Create trainable weight variable for this layer.
    self.kernel = self.add_weight(name='kernel',
                                 shape=(input_shape[1]),
                                 initializer='uniform',
                                 trainable=True)
    
  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.norm1(x)
    x = self.relu1(x)
    x = self.conv2(x)
    x = self.norm2(x)
    x = self.relu2(x)
    x = self.add(x, inputs) # with relu
    return self.relu3(x)

### Create trunk

In [0]:
def create_trunk(shape, nr_residual_layers):
  return keras.Sequential([
      layers.Conv2D(input_shape=shape),
      layers.BatchNormalization(),
      layers.ReLU(),
      Residual() for _ in range(nr_residual_layers)
  ])

### Create value head

In [5]:
def create_value_head(shape):
    return keras.Sequential([
        layers.Conv2D(1, kernel_size=(1,1)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(256),
        layers.ReLU(),
        layers.Dense(1, activation="tanh")
    ])

SyntaxError: ignored

## Create new model 

In [0]:
class Model:
  def __init__(self, trunk, value_head, policy_head):
    # Can be choosen by mode
    self.trunk = trunk
    self.value_head = value_head
    self.policy_head = policy_head
  
    # Can also be choosen by mode
    self.value_path = self.value_head(self.trunk)
    self.policy_path = self.policy_head(self.trunk)
    
  def save(self, file_name: str):
    pass
  
  def train(self, mode, X, y):
    pass
  
  def predict(self, mode, X):
    pass
    

  def create_new_model(shape=(5,5,7), nr_residual_layers=10) -> Model:
    trunk = create_trunk(shape, nr_residual_layers)
    value_head = create_value_head()
    policy_head = create_policy_head(shape)

    return Model(trunk, value_head, policy_head)

In [0]:
trunk = keras.Input(shape=(5,5,5))